<a href="https://colab.research.google.com/github/marco-siino/gpt_networking/blob/main/blueprint/CodeGemma_7B_IT_BluePrintToNS3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries and load the model.

In [ ]:
import os
import random
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

model_id = "cache/models--google--codegemma-7b-it/snapshots/078cdc51070553d1636d645c9a238f3b0914459a/"

#model_id = "google/gemma-7b-it"
dtype = torch.bfloat16

# I can decide which GPU to use on this node on Leonardo.
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.cuda.set_device(0)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=dtype,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
!nvidia-smi

Thu Nov 28 13:04:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM-64GB            On | 00000000:1D:00.0 Off |                    0 |
| N/A   43C    P0               77W / 466W|   4617MiB / 65536MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# Generate the Python code to perform the task.

In [ ]:
with open("projects/testbed-at-JNCA/blueprint/sample_blueprint.json") as f:
        sample_blueprint = f.read()
with open("projects/testbed-at-JNCA/blueprint/sample_ns3_code.cc") as f:
        sample_ns3_code = f.read()
with open("projects/testbed-at-JNCA/blueprint/test/0000_blueprint_nDevices_5_nGateways_5-radius-1.json") as f:
        test_blueprint = f.read()

In [ ]:
prompt = """
 Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT:

        SAMPLE BLUEPRINT:

        """+sample_blueprint+ """

        SAMPLE NS3 CODE:

        """+sample_ns3_code+"""

        TEST BLUEPRINT:

        """+test_blueprint+"""

        NS3 CODE:

        """
print(prompt)

In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(**input_ids, max_new_tokens=2000)
print(tokenizer.decode(outputs[0]))

/leonardo/home/userexternal/msiino00/clean-updated-env/lib/python3.10/site-packages/transformers/generation/utils.py:1788: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


<bos>
 Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT:

        SAMPLE BLUEPRINT:

        {
  "type": "configuration",
  "version": "1.0",
  "mqtt": {
    "broker": "<mqtt broker hostname>",
    "port": 8883,
    "client_id": "mqtt-db-logger",
    "username": "<username here>",
    "password": "<passowrd here>",
    "topics": [
      "application/+/device/+/event/#"
    ]
  },
  "mongodb": {
    "host": "<mongodb database hostname>",
    "port": 27017,
    "database": "Chirpstack-Supernova"
  },
  "network": {
    "id": "lorawan-network",
    "name": "LoRaWAN Network",
    "network_server": {
      "id": "network-server-1",
      "name": "ChirpStack Network Server",
      "address": "<network server addr>",
      "api_token": "<API token Here>"
    },
    "coverage": {
      "radius": 1
    },
    "gateways": [
      {
        "id": "gateway-1",
        "name": "LoRa Gateway 1",
        "